# Just Value New Orleans Gentrification Analysis
Trey Briggs 05/17/2023 6:30PM EST

In [64]:
import os 
os.chdir('C:/Users/brigg/Documents/JupyterNotebooks/JustValue')
# Load Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from zipfile import ZipFile 
from io import BytesIO
import requests
import json
import dotenv

For this assignment, I will be using

## Data Extraction

### US Census Dataset

In [65]:
dotenv.load_dotenv()

True

In [66]:
# Map variable names to their respective indices in the response
variable_mapping = {
    'B19013_001E': 'median_income',
    'B25064_001E': 'median_rent',
    'B01001_001E': 'total_pop',
    'B01001H_008E': 'male_white_20_24',
    'B01001H_009E': 'male_white_25_29',
    'B01001H_010E': 'male_white_30_34',
    'B01001H_023E': 'female_white_20_24',
    'B01001H_024E': 'female_white_25_29',
    'B01001H_025E': 'female_white_30_34',
}

In [67]:
# Pull variables from already created map for API request
variables = variable_mapping.keys()
variables_string = ','.join(variables)
variables_string

'B19013_001E,B25064_001E,B01001_001E,B01001H_008E,B01001H_009E,B01001H_010E,B01001H_023E,B01001H_024E,B01001H_025E'

In [68]:
# Construct the API request URL
base_url = "https://api.census.gov/data"
endpoint = "/{year}/acs/acs5"
census_key = os.getenv('census_key') # API key security

# Specify the variables and geographic area
variables = variables_string  # Vars
state = "22"  # Louisiana's FIPS code
county = "071"  # Orleans Parish's FIPS code
in_metropolitan_statistical_area = "35380"  # New Orleans-Metairie, LA Metro Area's CBSA code

# Define the range of years
start_year = 2013
end_year = 2021


census_df = pd.DataFrame()

# Loop through each year and make API requests
for year in range(start_year, end_year + 1):
    # Construct the request URL
    url = f"{base_url}{endpoint.format(year=year)}"
    params = {
        "get": variables,
        "for": "tract:*",
        "in" : "state:22 county:071",
        "key": census_key,
    }

    # Make the API request
    response = requests.get(url, params=params)

    # Process the response
    if response.status_code == 200:
        # Process and work with the data as needed for each year
        data = response.json()
        
        temp_df = pd.DataFrame(data[1:], columns=data[0])
        temp_df['year'] = year
        
        census_df = census_df.append(temp_df)
    else:
        print("Error:", response.text)
        print(url)

# Reset the index of the resulting DataFrame
census_df.reset_index(drop=True, inplace=True)

variable_mapping = {
    'B19013_001E': 'median_income',
    'B25064_001E': 'median_rent',
    'B01001_001E': 'total_pop',
    'B01001H_008E': 'male_white_20_24',
    'B01001H_009E': 'male_white_25_29',
    'B01001H_010E': 'male_white_30_34',
    'B01001H_023E': 'female_white_20_24',
    'B01001H_024E': 'female_white_25_29',
    'B01001H_025E': 'female_white_30_34',
    'state':'state',
    'county':'county',
}

#rename the column data frames
census_df.rename(columns=variable_mapping, inplace=True)

# Display the resulting DataFrame
census_df.head()

C:\Users\brigg\AppData\Local\Temp\ipykernel_27576\113462738.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  census_df = census_df.append(temp_df)
C:\Users\brigg\AppData\Local\Temp\ipykernel_27576\113462738.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  census_df = census_df.append(temp_df)
C:\Users\brigg\AppData\Local\Temp\ipykernel_27576\113462738.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  census_df = census_df.append(temp_df)
C:\Users\brigg\AppData\Local\Temp\ipykernel_27576\113462738.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  census_df = census_df.append(temp_df)
C:\Users\brigg\AppData\Local\Temp\ipyker

,median_income,median_rent,total_pop,male_white_20_24,male_white_25_29,male_white_30_34,female_white_20_24,female_white_25_29,female_white_30_34,state,county,tract,year
0,86182,1023,1904,15,101,35,23,49,44,22,071,009000,2013
1,51477,971,1675,32,98,12,127,47,80,22,071,009600,2013
2,65236,1271,2120,45,144,95,63,176,53,22,071,010100,2013
3,46875,1011,1232,14,34,62,3,64,57,22,071,010600,2013
4,81406,1433,1315,48,33,85,49,91,38,22,071,011500,2013


In [69]:
#remove columsn where total pop is zero
census_df.total_pop = census_df.total_pop.astype(int)
census_df = census_df.loc[census_df['total_pop'] != 0]

In [70]:
#minor data cleansing needed at this point
census_df2 = census_df

start_col = 'median_income'
end_col = 'female_white_30_34'
census_df2.loc[:, start_col:end_col] = census_df2.loc[:, start_col:end_col].apply(pd.to_numeric, errors='coerce').astype(int)

#sum columns and divide by total to get the prop target used in our index
start_col = 'male_white_20_24'
end_col = 'female_white_30_34'
census_df2['prop_white_20_34'] = census_df2.loc[:, start_col:end_col].sum(axis=1)/census_df2.total_pop


#convert to string
census_df2.state = census_df2.state.astype(str)
census_df2.county = census_df2.county.astype(str)
census_df2.tract = census_df2.tract.astype(str)

#join to creat 11 digit fips code for matching
census_df2['TRACT'] = census_df2['state'].str.cat([census_df2['county'], census_df2['tract']], sep='')
census_df2 = census_df2.drop(['state', 'county', 'tract'], axis = 1)

census_df2.head()

,median_income,median_rent,total_pop,male_white_20_24,male_white_25_29,male_white_30_34,female_white_20_24,female_white_25_29,female_white_30_34,year,prop_white_20_34,TRACT
0,86182,1023,1904,15,101,35,23,49,44,2013,0.140231,22071009000
1,51477,971,1675,32,98,12,127,47,80,2013,0.236418,22071009600
2,65236,1271,2120,45,144,95,63,176,53,2013,0.271698,22071010100
3,46875,1011,1232,14,34,62,3,64,57,2013,0.189935,22071010600
4,81406,1433,1315,48,33,85,49,91,38,2013,0.261597,22071011500


In [71]:
#We will match on these tracts later
tracts = census_df2.TRACT.unique()

In [72]:
# Filter out string columns
numeric_cols = census_df2.select_dtypes(exclude='object').columns
numeric_cols
census_df2.TRACT = census_df2.TRACT.astype(np.int64)

Index(['median_income', 'median_rent', 'total_pop', 'male_white_20_24',
       'male_white_25_29', 'male_white_30_34', 'female_white_20_24',
       'female_white_25_29', 'female_white_30_34', 'year', 'prop_white_20_34'],
      dtype='object')

In [73]:
census_df2 = census_df2.loc[(census_df2[numeric_cols] >= 0).all(axis=1)]

In [74]:
census_df2

,median_income,median_rent,total_pop,male_white_20_24,male_white_25_29,male_white_30_34,female_white_20_24,female_white_25_29,female_white_30_34,year,prop_white_20_34,TRACT
0,86182,1023,1904,15,101,35,23,49,44,2013,0.140231,22071009000
1,51477,971,1675,32,98,12,127,47,80,2013,0.236418,22071009600
2,65236,1271,2120,45,144,95,63,176,53,2013,0.271698,22071010100
3,46875,1011,1232,14,34,62,3,64,57,2013,0.189935,22071010600
4,81406,1433,1315,48,33,85,49,91,38,2013,0.261597,22071011500
...,...,...,...,...,...,...,...,...,...,...,...,...
1598,45119,888,1608,0,25,7,9,5,0,2021,0.028607,22071014101
1599,36645,1087,992,0,38,13,7,22,0,2021,0.080645,22071014102
1600,99063,1347,1748,2,20,41,3,70,99,2021,0.134439,22071014200
1601,18611,816,1778,6,5,5,0,9,5,2021,0.016873,22071014300


### FHFA Dataset|

In [75]:
# Load fhfa dataset
fhfa_path = 'https://www.fhfa.gov/DataTools/Documents/UAD-Data-Files/UADAggs_tract.zip'

# by default pandas uses zip as compression
fhfa_df = pd.read_csv(fhfa_path)
fhfa_df.head()

,SOURCE,SERIES,SERIESID,FREQUENCY,GEOLEVEL,GEONAME,STATEPOSTAL,STATEFIPS,FIPS,TRACT,METRO,PURPOSE,YEAR,QUARTER,CHARACTERISTIC,category,SUPPRESSED,VALUE
0,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,AL,1,1001,1001020100,NaN,Purchase,2013,5,No Characteristic,All Appraisals,1,NaN
1,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,AL,1,1001,1001020100,NaN,Refinance,2013,5,No Characteristic,All Appraisals,1,NaN
2,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,AL,1,1001,1001020100,NaN,Both,2013,5,No Characteristic,All Appraisals,0,16.0
3,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,AL,1,1001,1001020100,NaN,Purchase,2014,5,No Characteristic,All Appraisals,1,NaN
4,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,AL,1,1001,1001020100,NaN,Refinance,2014,5,No Characteristic,All Appraisals,1,NaN


In [76]:
#select Lousiana
fhfa_LA_df = fhfa_df.query("STATEFIPS==22")

fhfa_LA_df.head()

,SOURCE,SERIES,SERIESID,FREQUENCY,GEOLEVEL,GEONAME,STATEPOSTAL,STATEFIPS,FIPS,TRACT,METRO,PURPOSE,YEAR,QUARTER,CHARACTERISTIC,category,SUPPRESSED,VALUE
933356,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Purchase,2013,5,No Characteristic,All Appraisals,1,NaN
933357,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Refinance,2013,5,No Characteristic,All Appraisals,1,NaN
933358,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Both,2013,5,No Characteristic,All Appraisals,0,16.0
933359,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Purchase,2014,5,No Characteristic,All Appraisals,1,NaN
933360,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Refinance,2014,5,No Characteristic,All Appraisals,1,NaN


In [77]:
#select New Orleans 
fhfa_LA_df = fhfa_LA_df.query("PURPOSE =='Both'")
fhfa_LA_df.head()

,SOURCE,SERIES,SERIESID,FREQUENCY,GEOLEVEL,GEONAME,STATEPOSTAL,STATEFIPS,FIPS,TRACT,METRO,PURPOSE,YEAR,QUARTER,CHARACTERISTIC,category,SUPPRESSED,VALUE
933358,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Both,2013,5,No Characteristic,All Appraisals,0,16.0
933361,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Both,2014,5,No Characteristic,All Appraisals,0,21.0
933364,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Both,2015,5,No Characteristic,All Appraisals,0,15.0
933367,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Both,2016,5,No Characteristic,All Appraisals,0,26.0
933370,UAD,Count of Appraisals,COUNT,Quarterly,Tract,NaN,LA,22,22001,22001960101,NaN,Both,2017,5,No Characteristic,All Appraisals,0,26.0


In [89]:
#Create columns for each series to act as a variable
fhfa_pivot_df = fhfa_LA_df.pivot(index=['FIPS', 'TRACT','YEAR'],
                    columns=['SERIES'],
                    values='VALUE').reset_index()
fhfa_pivot_df = fhfa_pivot_df[fhfa_pivot_df['YEAR']!=2022]
fhfa_pivot_df.TRACT = fhfa_pivot_df.TRACT.astype(np.int64)

In [90]:
fhfa_clean = fhfa_pivot_df.dropna()

### NOLA Business Licenses Data
#### The convert to FIPS ended up taking way too long. 

In [106]:
# NOLA_df = pd.read_csv('https://data.nola.gov/api/views/hjcd-grvu/rows.csv?accessType=DOWNLOAD')

In [110]:
# NOLA_df.head()

,BusinessName,OwnerName,BusinessType,BusinessLicenseNumber,BusinessStartDate,Address,StreetNumber,StreetDirection,StreetName,StreetSuffix,...,Zip,PhoneNumber,MailAddress,MailSuite,MailCity,MailState,MailZip,Latitude,Longitude,Location
0,FACTUMS,"BULL INVESTIGATIONS, LLC",3801 - INVESTIGATION SERVICES,291590,05/24/2021,6735 E HERMES ST,6735,E,HERMES,ST,...,70126,5044139958,6735 E HERMES ST,NaN,NEW ORLEANS,LA,70126,30.023474,-90.015967,"(30.0234737941083, -90.0159674788499)"
1,"AG 2018, LLC","AG 2018, LLC",3309 - NONRESIDENTIAL PROPERTY MANAGERS,293819,12/14/2021,1626 ORETHA CASTLE HALEY BLVD,1626,NaN,ORETHA CASTLE HALEY,BLVD,...,70113,NaN,1626 ORETHA CASTLE HALEY BLVD,NaN,NEW ORLEANS,LA,70113,29.939877,-90.079936,"(29.9398769235327, -90.079936156175)"
2,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,2033 - JANITORIAL SERVICES,293860,10/07/2019,2139 HOLIDAY DR,2139,NaN,HOLIDAY,DR,...,70114,NaN,2139 HOLIDAY DR,NaN,NEW ORLEANS,LA,70114,29.932313,-90.013698,"(29.9323132108077, -90.0136980627568)"
3,A1 NEMT LLC,A1 NEMT LLC,2032 - SPECIAL NEEDS TRANSPORTATION,293822,03/07/2019,6235 ST. ANTHONY AVE,6235,NaN,ST. ANTHONY,AVE,...,70122,NaN,6235 ST. ANTHONY AVE,NaN,NEW ORLEANS,LA,70122,NaN,NaN,NaN
4,KAINTOCKS,ODUM LANE VENTURES LLC,1105 - FULL SVC RESTAURANTS (TABLE SERVICE),291807,02/15/2022,735 ST. JOSEPH ST,735,NaN,ST. JOSEPH,ST,...,70130,NaN,735 ST. JOSEPH ST,NaN,NEW ORLEANS,LA,70130,NaN,NaN,NaN


In [112]:
# #Extract year for aggregations
# from datetime import datetime

# def extract_year(date_string):
#     try:
#         # Parse the date string into a datetime object
#         date_obj = datetime.strptime(date_string, '%m/%d/%Y')
#         # Extract the year from the datetime object
#         year = date_obj.year
#         return year
#     except ValueError:
#         # Handle any invalid date strings or formatting errors
#         return None

# # Example usage
# NOLA_df['year'] = NOLA_df['BusinessStartDate'].apply(extract_year)
# NOLA_df.head()

,BusinessName,OwnerName,BusinessType,BusinessLicenseNumber,BusinessStartDate,Address,StreetNumber,StreetDirection,StreetName,StreetSuffix,...,PhoneNumber,MailAddress,MailSuite,MailCity,MailState,MailZip,Latitude,Longitude,Location,year
0,FACTUMS,"BULL INVESTIGATIONS, LLC",3801 - INVESTIGATION SERVICES,291590,05/24/2021,6735 E HERMES ST,6735,E,HERMES,ST,...,5044139958,6735 E HERMES ST,NaN,NEW ORLEANS,LA,70126,30.023474,-90.015967,"(30.0234737941083, -90.0159674788499)",2021
1,"AG 2018, LLC","AG 2018, LLC",3309 - NONRESIDENTIAL PROPERTY MANAGERS,293819,12/14/2021,1626 ORETHA CASTLE HALEY BLVD,1626,NaN,ORETHA CASTLE HALEY,BLVD,...,NaN,1626 ORETHA CASTLE HALEY BLVD,NaN,NEW ORLEANS,LA,70113,29.939877,-90.079936,"(29.9398769235327, -90.079936156175)",2021
2,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,2033 - JANITORIAL SERVICES,293860,10/07/2019,2139 HOLIDAY DR,2139,NaN,HOLIDAY,DR,...,NaN,2139 HOLIDAY DR,NaN,NEW ORLEANS,LA,70114,29.932313,-90.013698,"(29.9323132108077, -90.0136980627568)",2019
3,A1 NEMT LLC,A1 NEMT LLC,2032 - SPECIAL NEEDS TRANSPORTATION,293822,03/07/2019,6235 ST. ANTHONY AVE,6235,NaN,ST. ANTHONY,AVE,...,NaN,6235 ST. ANTHONY AVE,NaN,NEW ORLEANS,LA,70122,NaN,NaN,NaN,2019
4,KAINTOCKS,ODUM LANE VENTURES LLC,1105 - FULL SVC RESTAURANTS (TABLE SERVICE),291807,02/15/2022,735 ST. JOSEPH ST,735,NaN,ST. JOSEPH,ST,...,NaN,735 ST. JOSEPH ST,NaN,NEW ORLEANS,LA,70130,NaN,NaN,NaN,2022


In [121]:
# NOLA_filtered = NOLA_df[(NOLA_df["year"] >= 2013) & (NOLA_df["year"] <= 2021)]
# NOLA_filtered

,BusinessName,OwnerName,BusinessType,BusinessLicenseNumber,BusinessStartDate,Address,StreetNumber,StreetDirection,StreetName,StreetSuffix,...,PhoneNumber,MailAddress,MailSuite,MailCity,MailState,MailZip,Latitude,Longitude,Location,year
0,FACTUMS,"BULL INVESTIGATIONS, LLC",3801 - INVESTIGATION SERVICES,291590,05/24/2021,6735 E HERMES ST,6735,E,HERMES,ST,...,5044139958,6735 E HERMES ST,NaN,NEW ORLEANS,LA,70126,30.023474,-90.015967,"(30.0234737941083, -90.0159674788499)",2021
1,"AG 2018, LLC","AG 2018, LLC",3309 - NONRESIDENTIAL PROPERTY MANAGERS,293819,12/14/2021,1626 ORETHA CASTLE HALEY BLVD,1626,NaN,ORETHA CASTLE HALEY,BLVD,...,NaN,1626 ORETHA CASTLE HALEY BLVD,NaN,NEW ORLEANS,LA,70113,29.939877,-90.079936,"(29.9398769235327, -90.079936156175)",2021
2,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,2033 - JANITORIAL SERVICES,293860,10/07/2019,2139 HOLIDAY DR,2139,NaN,HOLIDAY,DR,...,NaN,2139 HOLIDAY DR,NaN,NEW ORLEANS,LA,70114,29.932313,-90.013698,"(29.9323132108077, -90.0136980627568)",2019
3,A1 NEMT LLC,A1 NEMT LLC,2032 - SPECIAL NEEDS TRANSPORTATION,293822,03/07/2019,6235 ST. ANTHONY AVE,6235,NaN,ST. ANTHONY,AVE,...,NaN,6235 ST. ANTHONY AVE,NaN,NEW ORLEANS,LA,70122,NaN,NaN,NaN,2019
6,SWEET DUPUY LLC,SWEET DUPUY LLC,3203 - INTERIOR DESIGN SERVICES,291765,10/12/2016,1316 VALENCE ST,1316,NaN,VALENCE,ST,...,NaN,1316 VALENCE ST,NaN,NEW ORLEANS,LA,70115,29.924309,-90.105694,"(29.9243091937069, -90.1056939504683)",2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12002,HOLLYWOOD CELEBRATION LOUNGE,HOLLYWOOD CELEBRATION LOUNGE LLC,1126 - DRINKING PLACES (ALCOHOLIC BEVERAGES),308022,05/16/2013,1535 N CLAIBORNE AVE,1535,N,CLAIBORNE,AVE,...,NaN,1221 N ROMAN ST,NaN,NEW ORLEANS,LA,70116,NaN,NaN,NaN,2013
12003,SAINT HUBERT INC.,SAINT HUBERT INC.,1160 - ELECTRONIC SHOPPING/MAIL-ORDER HOUSES,308051,10/20/2016,1745 LAKESHORE DR,1745,NaN,LAKESHORE,DR,...,NaN,1745 LAKESHORE DR,NaN,NEW ORLEANS,LA,70122,30.030622,-90.076828,"(30.0306218485828, -90.0768278605529)",2016
12005,EVERSAINT SALON,ELEMENT BEAUTY BAR LLC,3338 - BEAUTY SALONS,308023,08/15/2019,1515 ST CHARLES AVE,1515,NaN,ST CHARLES,AVE,...,NaN,1515 ST CHARLES AVE,STE 101,NEW ORLEANS,LA,70130,29.938839,-90.075951,"(29.9388391889167, -90.0759509099456)",2019
12006,"BLUEGREEN RESORTS MANAGEMENT, INC.","BLUEGREEN RESORTS MANAGEMENT, INC.",3305 - LESSOR OF RESIDENTIAL BLDGS/DWELLINGS,308087,04/23/2018,144 ELK PL,144,NaN,ELK,PL,...,NaN,144 ELK PL,NaN,NEW ORLEANS,LA,70112,29.955266,-90.073753,"(29.95526621251, -90.073753361932)",2018


In [155]:
# import requests

# def get_census_tract(row, api_key):
#     street_number = row["StreetNumber"]
#     street_name = row["StreetName"]
#     street_suffix = row["StreetSuffix"]
#     city = row["City"]
#     state = row["State"]
#     zip_code = row["Zip"]

#     address = f"{street_number} {street_name} {street_suffix}, {city}, {state} {zip_code}"

#     url = f"https://geocoding.geo.census.gov/geocoder/geographies/address?street={street_number}+{street_name}+{street_suffix}&city={city}&state={state}&zip={zip_code}&benchmark=Public_AR_Census2020&vintage=Census2020_Census2020&layers=8&format=json"
#     headers = {
#         "Accept": "application/json",
#         "Content-Type": "application/json",
#     }
#     # params = {
#     #     "key": census_key,
#     # }

#     response = requests.get(url, headers=headers, params=params)

#     if response.status_code == 200:
#         data = response.json()
#         address_matches = data.get("result", {}).get("addressMatches", [])
#         if address_matches:
#             census_tract = data["result"]["addressMatches"][0]["geographies"]["Census Block Groups"][0]["GEOID"]
#         else:
#             census_tract = "No Match"
#         return census_tract
#     else:
#         print("Error:", response.status_code)
#         return None

# # Example usage
# api_key = "YOUR_API_KEY"
# NOLA_filtered["census_tract"] = NOLA_filtered.apply(get_census_tract, api_key=census_key, axis=1)

BusinessName                                           FACTUMS
OwnerName                             BULL INVESTIGATIONS, LLC
BusinessType                     3801 - INVESTIGATION SERVICES
BusinessLicenseNumber                                   291590
BusinessStartDate                                   05/24/2021
Address                                       6735 E HERMES ST
StreetNumber                                              6735
StreetDirection                                              E
StreetName                                              HERMES
StreetSuffix                                                ST
Suite                                                      NaN
City                                               NEW ORLEANS
State                                                       LA
Zip                                                      70126
PhoneNumber                                         5044139958
MailAddress                                   6735 E HE

KeyboardInterrupt: 

In [154]:
NOLA_filtered

,BusinessName,OwnerName,BusinessType,BusinessLicenseNumber,BusinessStartDate,Address,StreetNumber,StreetDirection,StreetName,StreetSuffix,...,PhoneNumber,MailAddress,MailSuite,MailCity,MailState,MailZip,Latitude,Longitude,Location,year
0,FACTUMS,"BULL INVESTIGATIONS, LLC",3801 - INVESTIGATION SERVICES,291590,05/24/2021,6735 E HERMES ST,6735,E,HERMES,ST,...,5044139958,6735 E HERMES ST,NaN,NEW ORLEANS,LA,70126,30.023474,-90.015967,"(30.0234737941083, -90.0159674788499)",2021
1,"AG 2018, LLC","AG 2018, LLC",3309 - NONRESIDENTIAL PROPERTY MANAGERS,293819,12/14/2021,1626 ORETHA CASTLE HALEY BLVD,1626,NaN,ORETHA CASTLE HALEY,BLVD,...,NaN,1626 ORETHA CASTLE HALEY BLVD,NaN,NEW ORLEANS,LA,70113,29.939877,-90.079936,"(29.9398769235327, -90.079936156175)",2021
2,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,ANNIE'S COMPLETE HOUSE CLEANING & JANITORIAL S...,2033 - JANITORIAL SERVICES,293860,10/07/2019,2139 HOLIDAY DR,2139,NaN,HOLIDAY,DR,...,NaN,2139 HOLIDAY DR,NaN,NEW ORLEANS,LA,70114,29.932313,-90.013698,"(29.9323132108077, -90.0136980627568)",2019
3,A1 NEMT LLC,A1 NEMT LLC,2032 - SPECIAL NEEDS TRANSPORTATION,293822,03/07/2019,6235 ST. ANTHONY AVE,6235,NaN,ST. ANTHONY,AVE,...,NaN,6235 ST. ANTHONY AVE,NaN,NEW ORLEANS,LA,70122,NaN,NaN,NaN,2019
6,SWEET DUPUY LLC,SWEET DUPUY LLC,3203 - INTERIOR DESIGN SERVICES,291765,10/12/2016,1316 VALENCE ST,1316,NaN,VALENCE,ST,...,NaN,1316 VALENCE ST,NaN,NEW ORLEANS,LA,70115,29.924309,-90.105694,"(29.9243091937069, -90.1056939504683)",2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12002,HOLLYWOOD CELEBRATION LOUNGE,HOLLYWOOD CELEBRATION LOUNGE LLC,1126 - DRINKING PLACES (ALCOHOLIC BEVERAGES),308022,05/16/2013,1535 N CLAIBORNE AVE,1535,N,CLAIBORNE,AVE,...,NaN,1221 N ROMAN ST,NaN,NEW ORLEANS,LA,70116,NaN,NaN,NaN,2013
12003,SAINT HUBERT INC.,SAINT HUBERT INC.,1160 - ELECTRONIC SHOPPING/MAIL-ORDER HOUSES,308051,10/20/2016,1745 LAKESHORE DR,1745,NaN,LAKESHORE,DR,...,NaN,1745 LAKESHORE DR,NaN,NEW ORLEANS,LA,70122,30.030622,-90.076828,"(30.0306218485828, -90.0768278605529)",2016
12005,EVERSAINT SALON,ELEMENT BEAUTY BAR LLC,3338 - BEAUTY SALONS,308023,08/15/2019,1515 ST CHARLES AVE,1515,NaN,ST CHARLES,AVE,...,NaN,1515 ST CHARLES AVE,STE 101,NEW ORLEANS,LA,70130,29.938839,-90.075951,"(29.9388391889167, -90.0759509099456)",2019
12006,"BLUEGREEN RESORTS MANAGEMENT, INC.","BLUEGREEN RESORTS MANAGEMENT, INC.",3305 - LESSOR OF RESIDENTIAL BLDGS/DWELLINGS,308087,04/23/2018,144 ELK PL,144,NaN,ELK,PL,...,NaN,144 ELK PL,NaN,NEW ORLEANS,LA,70112,29.955266,-90.073753,"(29.95526621251, -90.073753361932)",2018


### Zillow Data
#### I'm going to use the ZHVI to see if the difference from the median house value to the ZHVI will increase preditive accuracy. 

In [183]:
url = 'https://files.zillowstatic.com/research/public_csvs/zhvi/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1684739249'
zillow_df = pd.read_csv(url)

In [186]:
zillow_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,208469.545294,...,860194.125359,858699.694776,855108.149163,851292.909154,848758.281718,845453.621302,838059.913020,828052.150813,819773.212443,815928.037995
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,135209.743813,...,287833.656571,286479.197849,284349.306082,282557.706686,281757.297709,280995.357778,281623.651834,282378.213360,283706.876634,284464.296826
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,107803.585034,...,284216.691741,285439.730049,285470.167172,285065.699920,284585.908859,283423.404229,281696.493625,280127.611574,279535.622215,279462.819123
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,144216.554561,...,480491.129894,480029.852101,475629.544967,469875.637125,463618.859924,456960.880046,451584.197172,447662.124182,446135.264108,445142.630588
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,215557.171867,...,905710.716610,898359.973900,886216.965364,875189.704464,866753.101029,859458.558851,852409.382100,847776.745169,847358.903493,850394.864590


In [185]:
zillow_df2 = zillow_df[(zillow_df['Metro']=='New Orleans-Metairie, LA')&(zillow_df['MunicipalCodeFIPS']==71)]
zillow_df2.head()

In [189]:
#Range of columns to melt
date_cols = zillow_df2.columns[9:]
date_cols

Index(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30', '2000-05-31',
       '2000-06-30', '2000-07-31', '2000-08-31', '2000-09-30', '2000-10-31',
       ...
       '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
       '2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30'],
      dtype='object', length=280)

In [198]:
#Move the months to their own row.
zillow_melted = pd.melt(zillow_df2, id_vars = 'MunicipalCodeFIPS', value_vars = date_cols, var_name='date', value_name='zhvi')
zillow_melted.head(1)

,MunicipalCodeFIPS,date,zhvi
0,71,2000-01-31,91782.759189


In [200]:
zillow_melted['date'] = pd.to_datetime(zillow_melted['date'])

In [207]:
# Group by year and calculate the average
zhvi_avg = zillow_melted.groupby(zillow_melted['date'].dt.year)['zhvi'].mean().reset_index()
zhvi_clean = zhvi_avg[zhvi_avg['date']>2012]
zhvi_clean = zhvi_clean.rename(columns={'date':'year'})

# Print the average values by year
print(zhvi_clean)

    year           zhvi
13  2013  140626.587803
14  2014  150538.200408
15  2015  164244.466053
16  2016  182126.404887
17  2017  184262.007452
18  2018  188764.322559
19  2019  208431.062201
20  2020  226043.058024
21  2021  251027.306379
22  2022  275290.824039
23  2023  266967.806212


### Merge Dataframes

In [81]:
merged_df = pd.merge(census_df2, fhfa_clean, left_on = ['TRACT','year'], right_on = ['TRACT', 'YEAR'])

In [205]:
merged_df.head()

,median_income,median_rent,total_pop,male_white_20_24,male_white_25_29,male_white_30_34,female_white_20_24,female_white_25_29,female_white_30_34,year,prop_white_20_34,TRACT,FIPS,YEAR,25% Quartile of Appraised Value,75% Quartile of Appraised Value,Count of Appraisals,Mean Appraised Value,Median Appraised Value
0,86182,1023,1904,15,101,35,23,49,44,2013,0.140231,22071009000,22071,2013,525000.0,1375000.0,29.0,929000.0,725000.0
1,51477,971,1675,32,98,12,127,47,80,2013,0.236418,22071009600,22071,2013,340000.0,450000.0,46.0,403500.0,395500.0
2,65236,1271,2120,45,144,95,63,176,53,2013,0.271698,22071010100,22071,2013,357000.0,680000.0,26.0,535700.0,487500.0
3,46875,1011,1232,14,34,62,3,64,57,2013,0.189935,22071010600,22071,2013,283000.0,530000.0,31.0,396400.0,350000.0
4,81406,1433,1315,48,33,85,49,91,38,2013,0.261597,22071011500,22071,2013,575000.0,890000.0,36.0,777700.0,690000.0


In [208]:
merged_df2 = pd.merge(merged_df, zhvi_clean, on='year')

In [209]:
merged_df2

,median_income,median_rent,total_pop,male_white_20_24,male_white_25_29,male_white_30_34,female_white_20_24,female_white_25_29,female_white_30_34,year,prop_white_20_34,TRACT,FIPS,YEAR,25% Quartile of Appraised Value,75% Quartile of Appraised Value,Count of Appraisals,Mean Appraised Value,Median Appraised Value,zhvi
0,86182,1023,1904,15,101,35,23,49,44,2013,0.140231,22071009000,22071,2013,525000.0,1375000.0,29.0,929000.0,725000.0,140626.587803
1,51477,971,1675,32,98,12,127,47,80,2013,0.236418,22071009600,22071,2013,340000.0,450000.0,46.0,403500.0,395500.0,140626.587803
2,65236,1271,2120,45,144,95,63,176,53,2013,0.271698,22071010100,22071,2013,357000.0,680000.0,26.0,535700.0,487500.0,140626.587803
3,46875,1011,1232,14,34,62,3,64,57,2013,0.189935,22071010600,22071,2013,283000.0,530000.0,31.0,396400.0,350000.0,140626.587803
4,81406,1433,1315,48,33,85,49,91,38,2013,0.261597,22071011500,22071,2013,575000.0,890000.0,36.0,777700.0,690000.0,140626.587803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,74695,1276,1103,2,4,18,5,9,45,2021,0.075249,22071013600,22071,2021,325000.0,525000.0,17.0,428100.0,440000.0,251027.306379
998,26683,879,2711,0,4,0,0,2,0,2021,0.002213,22071013800,22071,2021,189000.0,338000.0,19.0,292700.0,300000.0,251027.306379
999,45119,888,1608,0,25,7,9,5,0,2021,0.028607,22071014101,22071,2021,353000.0,585000.0,23.0,482100.0,474000.0,251027.306379
1000,99063,1347,1748,2,20,41,3,70,99,2021,0.134439,22071014200,22071,2021,445000.0,665000.0,62.0,588500.0,582000.0,251027.306379


### Plotting

In [104]:
# Read the shapefile for New Orleans tracts
shapefile_path = 'tl_rd22_22071_faces.shp'
tracts_gdf = gpd.read_file(shapefile_path)
tracts_gdf 

,TFID,STATEFP20,COUNTYFP20,TRACTCE20,BLKGRPCE20,BLOCKCE20,SUFFIX1CE,ZCTA5CE20,UACE20,PUMACE20,...,METDIVFP,CNECTAFP,NECTAFP,NCTADVFP,LWFLAG,OFFSET,ATOTAL,INTPTLAT,INTPTLON,geometry
0,260383227,22,071,003308,3,3004,NaN,70122,62677,02401,...,NaN,NaN,NaN,NaN,L,N,22648,+29.9935222,-90.0676062,"POLYGON ((-90.06828 29.99423, -90.06777 29.994..."
1,260762213,22,071,003308,3,3000,NaN,70122,62677,02401,...,NaN,NaN,NaN,NaN,L,N,28114,+29.9991398,-90.0655345,"POLYGON ((-90.06834 29.99948, -90.06712 30.000..."
2,260383235,22,071,003308,3,3002,NaN,70122,62677,02401,...,NaN,NaN,NaN,NaN,L,N,42106,+29.9975219,-90.0677972,"POLYGON ((-90.06824 29.99937, -90.06647 29.997..."
3,260383239,22,071,003308,3,3001,NaN,70122,62677,02401,...,NaN,NaN,NaN,NaN,L,N,51454,+29.9964317,-90.0636121,"POLYGON ((-90.06538 29.99681, -90.06497 29.997..."
4,266058279,22,071,002300,1,1027,NaN,70117,62677,02402,...,NaN,NaN,NaN,NaN,L,N,611,+29.9849761,-90.0581465,"POLYGON ((-90.05823 29.98521, -90.05819 29.985..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16519,267942294,22,071,008400,2,2006,NaN,70113,62677,02402,...,NaN,NaN,NaN,NaN,L,N,7905,+29.9349862,-90.0824474,"POLYGON ((-90.08327 29.93473, -90.08228 29.935..."
16520,267942298,22,071,009000,1,1005,NaN,70115,62677,02403,...,NaN,NaN,NaN,NaN,L,N,8223,+29.9300233,-90.0858425,"POLYGON ((-90.08615 29.93075, -90.08583 29.930..."
16521,267942300,22,071,013501,1,1002,NaN,70116,62677,02403,...,NaN,NaN,NaN,NaN,L,N,3144,+29.9599796,-90.0679861,"POLYGON ((-90.06824 29.96034, -90.06757 29.959..."
16522,267942302,22,071,011600,2,2012,NaN,70118,62677,02403,...,NaN,NaN,NaN,NaN,L,N,5101,+29.9243180,-90.1204017,"POLYGON ((-90.12046 29.92480, -90.12000 29.924..."


In [103]:
#tracts_gdf.TRACTCE20 = tracts_gdf.TRACTCE20.astype(np.int64)
# Merge census data with the geospatial data
merged_gdf = tracts_gdf.merge(merged_df, left_on='TRACTCE20', right_on='TRACT', how='left')

merged_gdf



ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
#plot
fig, ax = plt.subplots(1, figsize=(8, 8))
plt.xticks(rotation=90)

merged_gdf.plot(column="prop_white_20_34", cmap="Reds", linewidth=0.4, 
                   ax=ax, edgecolor=".4")
bar_info = plt.cm.ScalarMappable(cmap="Reds", norm=plt.Normalize(vmin=0, vmax=100))
bar_info._A = []
cbar = fig.colorbar(bar_info)